In [2]:
import awkward as ak

from coffea import processor, hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

import numpy as np

from Tools.objects import * #if there is an issue after pulling tools remove ../
from Tools.basic_objects import *
from Tools.cutflow import *
from Tools.config_helpers import *
from Tools.triggers import *
from Tools.btag_scalefactors import *
from Tools.ttH_lepton_scalefactors import *
from Tools.lepton_scalefactors import *
from klepto.archives import dir_archive
from Tools.samples import * # fileset_2018 #, fileset_2018_small
from processor.std_acumulators import *

In [5]:
tW_file = fileset_2018['tW_scattering']
ttbar_file = fileset_2018['ttbar2l']
WW_file = fileset_2018['WW']
WZ_file = fileset_2018['WZ']
DY_file = fileset_2018['DY']

In [6]:
jet = getJets(tW_file, minPt=0, pt_var='pt_nom')
jet = jet[ak.argsort(jet.pt_nom, ascending=False)]
jet = jet[~match(jet, muon, deltaRCut=0.4)]
jet = jet[~match(jet, electron, deltaRCut=0.4)]
light     = getBTagsDeepFlavB(jet, year=self.year, invert=True)
fw        = getFwdJet(light)

AttributeError: 'list' object has no attribute 'Jet'

In [120]:
ev = NanoEventsFactory.from_root(ttbar_file[0], schemaclass=NanoAODSchema).events()

In [121]:
ttbar_ev

<NanoEventsArray [<event 1:5723:5722009>, ... ] type='179226 * event'>

In [122]:
jet = getJets(ev, minPt=0, pt_var='pt_nom')
jet = jet[ak.argsort(jet.pt_nom, ascending=False)]
light     = getBTagsDeepFlavB(jet, year=2018, invert=True)
fw        = getFwdJet(light)

In [123]:
weight = Weights(len(ev))
weight.add("weight", ev.weight*cfg['lumi'][2018])

In [124]:
muon = Collections(ev, "Muon", "tight").get()
dimuon = choose(muon, 2)
electron = Collections(ev, "Electron", "tight").get()
dielectron = choose(electron,2)
lepton = ak.concatenate([muon, electron], axis=1)

In [127]:
event_selection = (ak.num(lepton, axis=1) > 10 & (ak.num(lepton, axis = 1) <=10))

In [128]:
fw[event_selection].pt

<Array [[], [69.2], [], ... [], [85], [142]] type='113095 * var * float32[parame...'>

In [112]:
ak.flatten(fw[event_selection].pt, axis=1)

<Array [78.1, 75.1, 125, ... 48.8, 77.4, 256] type='192096 * float32[parameters=...'>

In [108]:
ak.flatten(ak.ones_like(fw[event_selection].pt)*weight.weight()[event_selection])

<Array [0.452, 0.452, 0.452, ... 0.452, 0.452] type='192096 * float64'>

In [93]:
abs(dimuon.mass-91.2)>15

<Array [[], [], [], [], ... [], [], [], []] type='441337 * var * bool'>

In [95]:
(ak.any(abs(dimuon.mass-91.2)>15, axis=1) & ak.any(abs(dielectron.mass-91.2)>15, axis=1))

<Array [False, False, False, ... False, False] type='441337 * bool'>

In [90]:
ak.num(lepton, axis=1) >= 1

<Array [False, True, False, ... True, True] type='441337 * bool'>

In [96]:
fw.pt>40

<Array [[True], [], [True], ... [], [True], []] type='441337 * var * bool'>